In [2]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon


In [3]:
BU_df = pd.read_csv('lat_lon_BU_data.csv')
#modifications
BU_df.loc[BU_df['Plant Name'] == 'Tembusu Cogen','lat'] = 1.2775
BU_df.loc[BU_df['Plant Name'] == 'Tembusu Cogen','lon'] = 103.66673
BU_df = BU_df.loc[BU_df['Technology'] == 'Coal']
BU_df = BU_df.astype({'lat': 'float32','lon': 'float32'})
BU_df.reset_index(inplace = True)
BU_df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [4]:
WRI_df = pd.read_csv('global_power_plant_database_v_1_3/global_power_plant_database.csv')

/home/emfreese/anaconda3/envs/grid_mod/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
shapefile = gpd.read_file("IPCC-WGI-reference-regions-v4_shapefile/IPCC-WGI-reference-regions-v4.shp")


In [6]:
WRI_df = WRI_df.loc[WRI_df['primary_fuel'] == 'Coal']

In [7]:
WRI_df.reset_index(inplace = True)

In [8]:
shapefile.reset_index(inplace = True)

In [9]:
shapefile.rename(columns = {'index':'number'}, inplace = True)

In [10]:
region_list = pd.DataFrame(columns = ['acronym','name','geometry','number'])
for idx,lat,lon in zip(WRI_df.index, WRI_df['latitude'], WRI_df['longitude']):
    #print(idx, lat, lon)
    region_list.loc[idx,'acronym'] = np.squeeze(shapefile.loc[shapefile['geometry'].contains(Point(lon, lat)), 'Acronym']) #lon, lat ordering
    region_list.loc[idx, 'name'] = np.squeeze(shapefile.loc[shapefile['geometry'].contains(Point(lon, lat)), 'Name']) #lon, lat ordering
    region_list.loc[idx, 'geometry'] = np.squeeze(shapefile.loc[shapefile['geometry'].contains(Point(lon, lat)), 'geometry']) #lon, lat ordering
    region_list.loc[idx, 'number'] = np.squeeze(shapefile.loc[shapefile['geometry'].contains(Point(lon, lat)), 'number']) #lon, lat ordering

In [11]:
np.unique(region_list['name'])

array(['Bay-of-Bengal', 'C.North-America', 'Caribbean', 'E.Asia',
       'E.Australia', 'E.C.Asia', 'E.Europe', 'E.North-America',
       'E.Siberia', 'E.Southern-Africa', 'Madagascar', 'Mediterranean',
       'N.Central-America', 'N.E.South-America', 'N.Europe',
       'N.Pacific-Ocean', 'N.W.North-America', 'N.W.South-America',
       'New-Zealand', 'Russian-Arctic', 'Russian-Far-East', 'S.Asia',
       'S.Australia', 'S.Central-America', 'S.E.Asia',
       'S.E.South-America', 'S.Indic-Ocean', 'S.W.South-America',
       'Sahara', 'Tibetan-Plateau', 'W.C.Asia', 'W.North-America',
       'W.Siberia', 'W.Southern-Africa', 'West&Central-Europe'],
      dtype=object)

In [12]:
np.unique(region_list['acronym'])

array(['BOB', 'CAR', 'CNA', 'EAS', 'EAU', 'ECA', 'EEU', 'ENA', 'ESAF',
       'ESB', 'MDG', 'MED', 'NCA', 'NES', 'NEU', 'NPO', 'NWN', 'NWS',
       'NZ', 'RAR', 'RFE', 'SAH', 'SAS', 'SAU', 'SCA', 'SEA', 'SES',
       'SIO', 'SWS', 'TIB', 'WCA', 'WCE', 'WNA', 'WSAF', 'WSB'],
      dtype=object)

In [22]:
ar6_coal_location = region_list.groupby(region_list['acronym']).first()['number']

In [23]:
ar6_coal_location = ar6_coal_location.drop(['BOB','NPO','SIO'])

In [24]:
ar6_coal_location.to_csv('ar6_coal_location.csv')